In [1]:
pip install trimesh

   ---------------------------------------- 0.0/704.8 kB ? eta -:--:--
   ---------------------------------------- 0.0/704.8 kB ? eta -:--:--
   -------------- ------------------------- 262.1/704.8 kB ? eta -:--:--
   ---------------------------- --------- 524.3/704.8 kB 882.6 kB/s eta 0:00:01
   ---------------------------- --------- 524.3/704.8 kB 882.6 kB/s eta 0:00:01
   -------------------------------------- 704.8/704.8 kB 809.3 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import trimesh

# Explanation:
# - numpy: for handling arrays and numerical operations.
# - tensorflow: to build the neural network models for GANs.
# - matplotlib: for visualizing results.
# - trimesh: for rendering 3D voxel data into visualizations.


# 1. Importing Libraries

In this step, we import the following libraries:

- **NumPy**: Handles arrays and numerical operations, essential for data manipulation.
- **TensorFlow**: Used to build and train the GAN model, including the generator and discriminator networks.
- **Matplotlib**: Helps visualize training results and generated outputs.
- **Trimesh**: Assists in visualizing 3D voxel data, which is represented as grids of data points.

These libraries provide the foundational tools for implementing the GAN and visualizing the results effectively.


# 2. Loading and Preprocessing the Dataset



In [3]:
# Simulating a dataset (Replace this with actual dataset loading code for ModelNet10 or similar)
def generate_voxel_data(num_samples=100):
    """
    Generates synthetic 3D voxel grids for testing the GAN.
    Each voxel grid is a cube of size 32x32x32 with a random smaller cube inside.

    Parameters:
    - num_samples: Number of samples to generate.

    Returns:
    - numpy array of shape (num_samples, 32, 32, 32).
    """
    voxel_data = []
    for _ in range(num_samples):
        # Initialize an empty grid
        cube = np.zeros((32, 32, 32))

        # Randomly position a smaller cube within the grid
        x, y, z = np.random.randint(8, 24, size=3)
        cube[x:x+8, y:y+8, z:z+8] = 1  # Set the smaller cube to 1

        voxel_data.append(cube)

    return np.array(voxel_data)

# Generate dataset
voxel_faces = generate_voxel_data(num_samples=100)
print("Dataset Shape:", voxel_faces.shape)

# Normalize voxel data
voxel_faces = voxel_faces.astype('float32') / 1.0


Dataset Shape: (100, 32, 32, 32)


# 2. Loading and Preprocessing the Dataset

Here, we generate a synthetic dataset of 3D voxel grids for testing the GAN. Each voxel grid represents a cube of size `32×32×32`. Inside this grid, we randomly position smaller cubes (filled with ones) to simulate 3D shapes.

Key steps:
1. **Synthetic Dataset Generation**:
   - `np.zeros((32, 32, 32))`: Creates a 3D grid initialized with zeros (empty voxels).
   - Randomly selects coordinates for a smaller cube using `np.random.randint`.
   - Sets the voxels in the smaller cube to `1` to mark them as filled.
2. **Normalization**:
   - Converts voxel values to `float32` and normalizes them to the range `[0, 1]`.

This synthetic data acts as the input for the discriminator during GAN training.


# 3. Building the Generator



In [4]:
def build_generator(latent_dim):
    """
    Builds the generator model for 3D voxel grids.

    Parameters:
    - latent_dim: Dimension of the input latent vector.

    Returns:
    - A compiled generator model.
    """
    model = tf.keras.Sequential([
        # Dense layer to project latent vector into a 3D representation
        layers.Dense(4*4*4*256, activation="relu", input_dim=latent_dim),
        layers.Reshape((4, 4, 4, 256)),  # Reshape to a 3D grid

        # Transposed convolution layers for upsampling
        # Adjusted strides to 2 to reach 32x32x32 output
        layers.Conv3DTranspose(128, kernel_size=4, strides=2, padding="same", activation="relu"),
        # Adjusted strides to 2 to reach 32x32x32 output
        layers.Conv3DTranspose(64, kernel_size=4, strides=2, padding="same", activation="relu"),
        # Adjusted strides to 2 to reach 32x32x32 output
        layers.Conv3DTranspose(32, kernel_size=4, strides=2, padding="same", activation="relu"),
        # Adjusted strides to 1, kernel_size to 3 to avoid increasing spatial dimensions beyond 32x32x32 and maintain padding
        layers.Conv3DTranspose(1, kernel_size=3, strides=1, padding="same", activation="sigmoid")
    ])
    return model

# Instantiate the generator
latent_dim = 100  # Dimension of the input noise vector
generator = build_generator(latent_dim)
generator.summary()

c:\Users\Rishu\AppData\Local\anaconda3\envs\rexxes\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 16384)          │     1,654,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 4, 4, 4, 256)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_transpose                │ (None, 8, 8, 8, 128)   │     2,097,280 │
│ (Conv3DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_transpose_1              │ (None, 16, 16, 16, 64) │       524,352 │
│ (Conv3DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_transpose_2              │ (None, 32, 32, 32, 32) │       131,104 │
│ (Conv3DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_transpose_3              │ (None, 32, 32, 32, 1)  │           865 │
│ (Conv3DTranspose)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,408,385 (16.82 MB)

 Trainable params: 4,408,385 (16.82 MB)

 Non-trainable params: 0 (0.00 B)

# 3. Building the Generator

The generator takes a random latent vector as input and outputs a 3D voxel grid. The architecture consists of:

1. **Dense Layer**:
   - Projects the latent vector into a higher-dimensional space.
   - The output is reshaped into a small 3D grid (`4×4×4×256`).

2. **Conv3DTranspose Layers**:
   - Perform transposed convolutions to upsample the small 3D grid step-by-step.
   - Each layer doubles the size of the grid while reducing the number of channels.
   - Final output size: `32×32×32×1`.

3. **Sigmoid Activation**:
   - Ensures that the output voxel values are in the range `[0, 1]`.

This generator model transforms random noise into structured 3D data.


# 4. Building the Discriminator



In [5]:
def build_discriminator():
    """
    Builds the discriminator model for 3D voxel grids.

    Returns:
    - A compiled discriminator model.
    """
    model = tf.keras.Sequential([
        layers.Conv3D(32, kernel_size=4, strides=2, padding="same", input_shape=(32, 32, 32, 1)),
        layers.LeakyReLU(0.2),
        layers.Conv3D(64, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(0.2),
        layers.Conv3D(128, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(0.2),
        layers.Flatten(),
        layers.Dense(1, activation="sigmoid")
    ])
    return model

# Instantiate the discriminator
discriminator = build_discriminator()
discriminator.summary()


c:\Users\Rishu\AppData\Local\anaconda3\envs\rexxes\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv3d (Conv3D)                 │ (None, 16, 16, 16, 32) │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 16, 16, 16, 32) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_1 (Conv3D)               │ (None, 8, 8, 8, 64)    │       131,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ (None, 8, 8, 8, 64)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_2 (Conv3D)               │ (None, 4, 4, 4, 128)   │       524,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_2 (LeakyReLU)       │ (None, 4, 4, 4, 128)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │         8,193 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 665,825 (2.54 MB)

 Trainable params: 665,825 (2.54 MB)

 Non-trainable params: 0 (0.00 B)

# 4. Building the Discriminator

The discriminator classifies input voxel grids as real or fake. Its architecture includes:

1. **Conv3D Layers**:
   - Extract 3D spatial features using convolutions.
   - Apply a LeakyReLU activation to retain information from small negative values.

2. **Flatten Layer**:
   - Flattens the 3D grid into a 1D vector for classification.

3. **Dense Layer**:
   - Outputs a single value indicating the probability of the input being real (`1`) or fake (`0`).
   - Uses a sigmoid activation for binary classification.

This discriminator helps the GAN learn by providing feedback on how realistic the generated voxel grids are.


# 5. Building the GAN


In [6]:
def build_gan(generator, discriminator):
    """
    Combines the generator and discriminator into a GAN model.
    """
    discriminator.compile(optimizer=tf.keras.optimizers.Adam(0.0002, 0.5), loss="binary_crossentropy", metrics=["accuracy"])
    discriminator.trainable = False  # Freeze discriminator for GAN training

    gan_input = layers.Input(shape=(latent_dim,))
    generated_voxel = generator(gan_input)
    gan_output = discriminator(generated_voxel)

    gan = tf.keras.Model(gan_input, gan_output)
    gan.compile(optimizer=tf.keras.optimizers.Adam(0.0002, 0.5), loss="binary_crossentropy")
    return gan

# Build and compile the GAN
gan = build_gan(generator, discriminator)
gan.summary()


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 32, 32, 32, 1)  │     4,408,385 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_1 (Sequential)       │ (None, 1)              │       665,825 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,074,210 (19.36 MB)

 Trainable params: 4,408,385 (16.82 MB)

 Non-trainable params: 665,825 (2.54 MB)

# 5. Building the GAN

The GAN combines the generator and discriminator into a single model. The process involves:

1. **Compiling the Discriminator**:
   - Optimized with `binary_crossentropy` loss and Adam optimizer.
   - Discriminator is frozen (weights are not updated) during GAN training.

2. **Defining GAN Workflow**:
   - Input: A latent vector (random noise).
   - Generator transforms the noise into a 3D voxel grid.
   - Discriminator evaluates the generated voxel grid.

3. **Compiling the GAN**:
   - Uses `binary_crossentropy` loss to measure how well the generator produces realistic outputs.

The GAN architecture enables the generator to improve by receiving feedback from the discriminator.


# 6. Training the GAN



In [7]:
def train_gan(generator, discriminator, gan, data, latent_dim, epochs=5000, batch_size=32):
    """
    Trains the GAN using the given dataset.
    """
    real = np.ones((batch_size, 1))  # Labels for real data
    fake = np.zeros((batch_size, 1))  # Labels for fake data

    for epoch in range(epochs):
        # Train Discriminator
        idx = np.random.randint(0, data.shape[0], batch_size)
        real_voxels = np.expand_dims(data[idx], axis=-1)

        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        fake_voxels = generator.predict(noise)

        d_loss_real = discriminator.train_on_batch(real_voxels, real)
        d_loss_fake = discriminator.train_on_batch(fake_voxels, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train Generator
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        g_loss = gan.train_on_batch(noise, real)

        if epoch % 500 == 0:
            # Directly access d_loss, as it's a scalar and doesn't need indexing
            print(f"Epoch {epoch}")

In [8]:
# Train the GAN
train_gan(generator, discriminator, gan, voxel_faces, latent_dim, epochs=10, batch_size=32)


# Adjust the epochs and batch_size as needed

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


AttributeError: 'NoneType' object has no attribute 'update_state'

# 6. Training the GAN

The GAN is trained using the following steps:

1. **Training the Discriminator**:
   - Separately trains on real and fake data.
   - Computes loss for real samples and fake samples, then averages them.

2. **Training the Generator**:
   - The generator creates fake samples.
   - The discriminator's feedback guides the generator to produce more realistic outputs.

3. **Loss Monitoring**:
   - Tracks the discriminator's (`d_loss`) and generator's (`g_loss`) losses during training.

By alternating training between the discriminator and generator, the GAN learns to produce realistic 3D voxel grids.


# 7. Generating and Visualizing 3D Faces



In [ ]:
def visualize_3d_voxel(voxel):
    """
    Visualizes a 3D voxel grid using trimesh.
    """
    mesh = trimesh.voxel.ops.matrix_to_marching_cubes(voxel)
    mesh.show()

# Generate a new 3D face
noise = np.random.normal(0, 1, (1, latent_dim))
generated_face = generator.predict(noise)
visualize_3d_voxel(generated_face[0, :, :, :, 0])

# Generate and visualize multiple 3D faces
num_faces = 5
generated_faces = generator.predict(np.random.normal(0, 1, (num_faces, latent_dim)))
for face in generated_faces:
    visualize_3d_voxel(face[:, :, :, 0])







1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


# 7. Generating and Visualizing 3D Faces

This step demonstrates how to use the trained generator to produce new 3D voxel grids:

1. **Generating 3D Faces**:
   - Input: A random latent vector.
   - Generator outputs a new 3D voxel grid.

2. **Visualization**:
   - Uses `Trimesh` to render the 3D voxel grid as a mesh.
   - The marching cubes algorithm converts the voxel data into a 3D mesh for visualization.

This step confirms that the generator has learned to produce plausible 3D shapes.


In [ ]:
# plot the loss
plt.plot(d_loss, label='Discriminator Loss')
plt.plot(g_loss, label='Generator Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

NameError: name 'd_loss' is not defined